# Проект: классификация

**Техническая задача для вас как для специалиста в Data Science:** построить модель машинного обучения, которая на основе предложенных характеристик клиента будет предсказывать, воспользуется он предложением об открытии депозита или нет.

[Датасет](https://lms.skillfactory.ru/assets/courseware/v1/dab91dc74eb3cb684755123d224d262b/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/bank_fin.zip)

[Ноутбук-шаблон](https://lms.skillfactory.ru/assets/courseware/v1/49573ace76275d21e852bc8fc5bb80bd/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/Project_4_ML.ipynb)

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from  sklearn.ensemble import IsolationForest
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing  import LabelEncoder
from sklearn import linear_model 
from sklearn import tree 
from sklearn import ensemble 
from sklearn import metrics 
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split 
from sklearn.feature_selection import SelectKBest, f_classif

## Часть 1. Знакомство с данными, обработка пропусков и выбросов

Данные о клиентах банка:

+ age (возраст);
+ job (сфера занятости);
+ marital (семейное положение);
+ education (уровень образования);
+ default (имеется ли просроченный кредит);
+ housing (имеется ли кредит на жильё);
+ loan (имеется ли кредит на личные нужды);
+ balance (баланс).

Данные, связанные с последним контактом в контексте текущей маркетинговой кампании:

+ contact (тип контакта с клиентом);
+ month (месяц, в котором был последний контакт);
+ day (день, в который был последний контакт);
+ duration (продолжительность контакта в секундах).

Прочие признаки:

+ campaign (количество контактов с этим клиентом в течение текущей кампании);
+ pdays (количество пропущенных дней с момента последней маркетинговой кампании до контакта в текущей кампании);
+ previous (количество контактов до текущей кампании)
+ poutcome (результат прошлой маркетинговой кампании).

И, разумеется, наша целевая переменная **deposit**, которая определяет, согласится ли клиент открыть депозит в банке. Именно её мы будем пытаться предсказать в данном кейсе.

### Задание 1

In [9]:
df = pd.read_csv('data/bank_fin.zip', sep = ';')

In [10]:
# исследуйте данные на предмет пропусков. Где есть пропущенные значения? Сколько их?
# ваш код
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11162 entries, 0 to 11161
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        11162 non-null  int64 
 1   job        11162 non-null  object
 2   marital    11162 non-null  object
 3   education  11162 non-null  object
 4   default    11162 non-null  object
 5   balance    11137 non-null  object
 6   housing    11162 non-null  object
 7   loan       11162 non-null  object
 8   contact    11162 non-null  object
 9   day        11162 non-null  int64 
 10  month      11162 non-null  object
 11  duration   11162 non-null  int64 
 12  campaign   11162 non-null  int64 
 13  pdays      11162 non-null  int64 
 14  previous   11162 non-null  int64 
 15  poutcome   11162 non-null  object
 16  deposit    11162 non-null  object
dtypes: int64(6), object(11)
memory usage: 1.4+ MB


Много категориальных признаков. Признак 'balance' имеет пропуски.

In [11]:
# количество пропусков в признаке 'balance'
df['balance'].isnull().sum()

25

### Задание 2

In [12]:
# есть ли в признаке job пропущенные значения? Возможно, они обозначены каким-то специальным словом?
# ваш код
df['job'].value_counts()

management       2566
blue-collar      1944
technician       1823
admin.           1334
services          923
retired           778
self-employed     405
student           360
unemployed        357
entrepreneur      328
housemaid         274
unknown            70
Name: job, dtype: int64

Значение признака 'unknown' является скрытым пропуском

### Задание 3

In [13]:
# преобразуйте признак balance таким образом, чтобы он корректно считывался, как вещественное число (float)
df['balance']

0         2 343,00 $ 
1            45,00 $ 
2         1 270,00 $ 
3         2 476,00 $ 
4           184,00 $ 
             ...     
11157         1,00 $ 
11158       733,00 $ 
11159        29,00 $ 
11160           0  $ 
11161           0  $ 
Name: balance, Length: 11162, dtype: object

In [14]:
# убираем знаки пробела и доллара, запятую меняем на точку
df['balance'] = df['balance'].apply(lambda x: str(x).replace(" ", "").replace("$", "").replace(",", "."))

In [15]:
# меняем признак на тип 'float'
df['balance'] = df['balance'].astype('float')

In [16]:
# вычислитм среднее значение по преобразованному в корректный вид признаку 'balance'
df['balance'].mean()

1529.1290293615875

### Задание 4

In [17]:
# обработайте пропуски в признаки balance , заменив их на медианные значения по данному признаку
# ваш код
df['balance'] = df['balance'].fillna(df['balance'].median())

In [18]:
# проверим на отсутствие пропусков в признаке
df['balance'].isna().sum()

0

In [19]:
# вычислитм среднее значение по преобразованному в корректный вид признаку 'balance'
df['balance'].mean()

1526.9360329690019

### Задание 5

In [20]:
# обработайте пропуски в категориальных признаках: job и education, заменив их на модальные значения
# ваш код

# скрытый пропуск имеет значение 'unknown', заменим его на numpy 'np.nan'
df['job'] = df['job'].apply(lambda x: np.nan if x == 'unknown' else x)
df['education'] = df['education'].apply(lambda x: np.nan if x == 'unknown' else x)

print('Количество пропусков в признаке job: {}'.format(df['job'].isna().sum()))
print('Количество пропусков в признаке education: {}'.format(df['education'].isna().sum()))

Количество пропусков в признаке job: 70
Количество пропусков в признаке education: 497


In [21]:
# заменим пропуски на модальное значения по каждому признаку
df['job'] = df['job'].fillna(df['job'].mode()[0])
df['education'] = df['education'].fillna(df['education'].mode()[0])

In [22]:
# проверим, что пропусков нет
print('Количество пропусков в признаке job: {}'.format(df['job'].isna().sum()))
print('Количество пропусков в признаке education: {}'.format(df['education'].isna().sum()))

Количество пропусков в признаке job: 0
Количество пропусков в признаке education: 0


### Задание 6

In [ ]:
# удалите все выбросы для признака balance
# ваш код

## Часть 2:  Разведывательный анализ

### Задание 1

In [ ]:
# изучите соотношение классов в ваших данных на предмет несбалансированности, проиллюстрируйте результат
# ваш код

### Задания 2 и 3

In [ ]:
#рассчитайте описательные статистики для количественных переменных, проинтерпретируйте результат
#ваш код

### Задания 4 и 5

In [ ]:
#рассчитайте описательные статистики для категориальных переменных, проинтерпретируйте результат
#ваш код
#постройте визуализации, иллюстрирующие результаты

### Задание 6

In [ ]:
# Узнайте, для какого статуса предыдущей маркетинговой кампании успех в текущей превалирует над количеством неудач.
# ваш код

### Задание 7

In [ ]:
# узнайте, в каком месяце чаще всего отказывались от предложения открыть депозит
# ваш код

### Задание 8

In [ ]:
# создайте возрастные группы и определите, в каких группах более склонны открывать депозит, чем отказываться от предложения

### Задания 9 и 10

In [ ]:
# постройте визуализации для открывших и неоткрывших депозит в зависимости от семейного статуса

In [ ]:
# постройте визуализации для открывших и неоткрывших депозит в зависимости от образования

In [ ]:
# постройте визуализации для открывших и неоткрывших депозит в зависимости от вида профессиональной занятости

### Задание 11

In [ ]:
# постройте сводную таблицу, чтобы определить люди с каким образованием и семейным статусом наиболее многочисленны
#(если рассматривать тех, кто открыл депозит)

## Часть 3: преобразование данных

### Задание 1

In [ ]:
# преобразуйте уровни образования

### Задания 2 и 3

In [ ]:
# преобразуйте бинарные переменные в представление из нулей и единиц

### Задание 4

In [ ]:
# создайте дамми-переменные

### Задания 5 и 6

In [ ]:
# постройте корреляционную матрицу и оцените данные на предмет наличия мультиколлинеарности

### Задания 7 и 8

In [3]:
X = df.drop(['deposit'], axis=1)
y = df['deposit']
 
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state = 42, test_size = 0.33)

In [ ]:
# рассчитайте необходимые показатели

### Задание 9

In [ ]:
# с помощью SelectKBest отберите 15 наиболее подходящих признаков

### Задание 10

In [ ]:
# нормализуйте данные с помощью minmaxsxaler

# Часть 4: Решение задачи классификации: логистическая регрессия и решающие деревья

### Задание 1

In [ ]:
# обучите логистическую регрессию и рассчитайте метрики качества

### Задания 2,3,4

In [ ]:
# обучите решающие деревья, настройте максимальную глубину

### Задание 5

In [ ]:
# подберите оптимальные параметры с помощью gridsearch

# Часть 5: Решение задачи классификации: ансамбли моделей и построение прогноза

### Задание 1

In [4]:
# обучите на ваших данных случайный лес

### Задания 2 и 3

In [ ]:
# используйте для классификации градиентный бустинг и сравните качество со случайным лесом

### Задание 4

In [ ]:
# объедините уже известные вам алгоритмы с помощью стекинга 

### Задание 5

In [ ]:
# оцените, какие признаки демонстрируют наибольшую  важность в модели градиентного бустинга

### Задания 6,7,8

In [ ]:
# реализуйте оптимизацию гиперпараметров с помощью Optuna